In [1]:
import os
import pandas as pd

# 데이터 확인 & 준비

### 간단한 무결성 확인
> 데이터가 온전하게 잘 있는가? 혹시 image 파일과 label 파일의 이름이 다르지 않는가?

In [2]:
def check_data(image_path, label_path):
    # 파일명 리스트 생성
    img_list, lbl_list = os.listdir(image_path), os.listdir(label_path)

    # 확장자 제거
    img_list = [i.split('.')[0] for i in img_list]
    lbl_list = [i.split('.')[0] for i in lbl_list]

    # 공통된 파일명 리스트 생성
    img_list = set(img_list)
    lbl_list = set(lbl_list)
    common_list = list(set(img_list) & set(lbl_list))

    print(f"이미지 개수: {len(img_list)} / 라벨 개수: {len(lbl_list)}")
    print(f"공통된 파일명 개수: {len(common_list)}")

# 함수 실행
for state in ['train', 'valid', 'test']:
    image_path = f"./dataset/{state}/images"
    label_path = f"./dataset/{state}/labels"
    check_data(image_path, label_path)

이미지 개수: 508 / 라벨 개수: 508
공통된 파일명 개수: 508
이미지 개수: 101 / 라벨 개수: 101
공통된 파일명 개수: 101
이미지 개수: 69 / 라벨 개수: 69
공통된 파일명 개수: 69


> 확장자가 다른 이미지가 발견됐다. 한 번 확인해보자. -> roboflow에서 다운받으니 다 통일됐다!

In [5]:
def check_extesion(image_path):
    # 파일명 리스트 생성
    img_list = os.listdir(image_path)

    # 확장자 리스트 생성
    img_ext = [i.split('.')[-1] for i in img_list]

    # 확장자별 개수
    img_ext = pd.Series(img_ext).value_counts()

    return img_ext

# 함수 실행
for state in ['train', 'valid', 'test']:
    image_path = f"./dataset/{state}/images"
    print(check_extesion(image_path))

jpg    508
Name: count, dtype: int64
jpg    101
Name: count, dtype: int64
jpg    69
Name: count, dtype: int64


### 파일의 클래스 분포 확인

In [ ]:
def check_distribution(label_path):
    # 라벨 리스트 생성
    lbl_list = os.listdir(label_path)

    # label 파일(txt)로부터 classID 추출
    class_list = []
    for lbl in lbl_list:
        with open(label_path+f"/{lbl}", 'r') as f:
            lines = f.readlines()
            for line in lines:
                # 현재 label은 yolo 형식이므로 첫번째 값이 class
                classID = line.split()[0]
                class_list.append(classID)
    
    return class_list

# 함수 실행
class_dict = {}
for state in ['train', 'valid', 'test']:
    label_path = f"./dataset/{state}/labels"
    class_list = check_distribution(label_path)
    class_dict[state] = class_list


train_df = pd.DataFrame(class_dict['train'], columns=['classID'])
train_df_counts = train_df.value_counts()

valid_df = pd.DataFrame(class_dict['valid'], columns=['classID'])
valid_df_counts = valid_df.value_counts()

test_df = pd.DataFrame(class_dict['test'], columns=['classID'])
test_df_counts = test_df.value_counts()

total_df = pd.concat([train_df_counts, valid_df_counts, test_df_counts],
                     axis=1, keys=['train', 'valid', 'test'])
total_df['sum'] = total_df.sum(axis=1)
total_df

,train,valid,test,sum
classID,,,,
0,439,81,95,615
1,200,54,41,295


### 객체가 여러 개 있는 일부 이미지의 bbox 그려보기

In [8]:
import os
import cv2
import numpy as np
    
# 경로 설정
image_dir = './dataset/images'
label_path = './dataset/labels'

image_list = ["V006_77_0_00_10_05_12_0_c14_20201224_0019_S01_1.jpg",
            "V006_77_0_00_10_05_12_0_c14_20201224_0004_S01_1.jpg",
            "V006_77_0_00_10_05_12_0_c18_20201221_0030_S01_1.JPG",
            "V006_77_0_00_10_05_12_0_c19_20201130_0032_S01_1.jpg",
            "V006_77_0_00_10_05_12_0_c19_20201130_0022_S01_2.jpg"]


for image in image_list:
    img_file = os.path.join(image_dir, image)
    # 이미지가 JPG일 경우
    if img_file.split('.')[-1] == 'JPG':
        lbl_file = os.path.join(label_path, image.replace('.JPG', '.txt'))
    # 이미지가 jpg일 경우
    elif img_file.split('.')[-1] == 'jpg':
        lbl_file = os.path.join(label_path, image.replace('.jpg', '.txt'))
    
    try:
        img = cv2.imread(img_file)
        if img is None:
            print(f"Error loading image: {img_file}")
            continue
    
        height, width = img.shape[:2]

        # label 파일(txt)로부터 bbox 정보 추출
        if os.path.exists(lbl_file):
            with open(lbl_file, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    classID, x, y, w, h = map(float, line.split())
                    x1 = int((x - w/2) * width)
                    y1 = int((y - h/2) * height)
                    x2 = int((x + w/2) * width)
                    y2 = int((y + h/2) * height)
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 5)
        else:
            print(f"Label file not found: {lbl_file}")
        
        # 결과 이미지 저장
        output_file = os.path.join('./output', f"bbox_{image}")
        if not os.path.exists('./output'):
            os.makedirs('./output')
        cv2.imwrite(output_file, img)
        print(f"Saved result image: {output_file}")

    except Exception as e:
        print(f"Error processing {image}: {e}")

print("All images processed")

Saved result image: ./output/bbox_V006_77_0_00_10_05_12_0_c14_20201224_0019_S01_1.jpg
Saved result image: ./output/bbox_V006_77_0_00_10_05_12_0_c14_20201224_0004_S01_1.jpg
Saved result image: ./output/bbox_V006_77_0_00_10_05_12_0_c18_20201221_0030_S01_1.JPG
Saved result image: ./output/bbox_V006_77_0_00_10_05_12_0_c19_20201130_0032_S01_1.jpg
Saved result image: ./output/bbox_V006_77_0_00_10_05_12_0_c19_20201130_0022_S01_2.jpg
All images processed
